In [1]:
from retinaface import RetinaFace
import cv2
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
img_path = "Ankit Kumar.jpg"
img = cv2.imread(img_path)
obj = RetinaFace.detect_faces(img_path)

In [3]:
for key in obj.keys():
    identify = obj[key]
    print(identify)
    facial_area = identify["facial_area"]
    cv2.rectangle(img,(facial_area[0],facial_area[1]),(facial_area[2],facial_area[3]),(255,255,0),3)

{'score': 0.9995902180671692, 'facial_area': [369, 338, 947, 1067], 'landmarks': {'right_eye': [511.90213, 621.2869], 'left_eye': [783.48425, 608.43036], 'nose': [638.28485, 744.85785], 'mouth_right': [533.3406, 883.25055], 'mouth_left': [762.4401, 871.33966]}}


In [4]:
from deepface import DeepFace

In [5]:
obj_df = DeepFace.find("img1.jpg", "students" ,model_name="ArcFace",
                         detector_backend="retinaface",distance_metric="cosine",enforce_detection= True,align = False)
embedding_objs = DeepFace.represent(img_path= "img1.jpg",model_name="ArcFace",
detector_backend="retinaface",enforce_detection= True,align = False)

24-10-01 01:09:03 - Searching img1.jpg in 11 length datastore
24-10-01 01:09:16 - find function duration 12.314710855484009 seconds


In [14]:
print(obj_df[0].identity)


0    students\Myself.jpg
1     students\Ankit.jpg
Name: identity, dtype: object


In [3]:
if isinstance(resp, dict):
    for key in resp:
        identity = resp[key]
        # Extract bounding box coordinates
        facial_area = identity["facial_area"]
        x1, y1, x2, y2 = facial_area
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extract and draw landmarks
        landmarks = identity["landmarks"]
        for landmark in landmarks.values():
            cv2.circle(img_rgb, (int(landmark[0]), int(landmark[1])), 2, (255, 0, 0), -1)
else:
    print("No faces detected.")


NameError: name 'cv2' is not defined

In [ ]:
import os
import cv2
import pandas as pd
from retinaface import RetinaFace
import face_recognition
from datetime import datetime
import numpy as np

def prepare_student_database(student_dir):
    student_db = {}
    for filename in os.listdir(student_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(student_dir, filename)
            name = os.path.splitext(filename)[0]
            try:
                # Load image using face_recognition
                image = face_recognition.load_image_file(filepath)
                encodings = face_recognition.face_encodings(image)
                if encodings:
                    student_db[name] = encodings[0]
                    print(f"Processed {name}'s image.")
                else:
                    print(f"No face found in {filename}, skipping.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return student_db

def detect_faces_in_classroom(classroom_image_path):
    try:
        # Detect faces using RetinaFace
        detections = RetinaFace.detect_faces(classroom_image_path)
        print(f"Detected {len(detections)} faces in the classroom image.")
        return detections
    except Exception as e:
        print(f"Error processing classroom image: {e}")
        return {}

def calculate_cosine_distance(emb1, emb2):
    emb1 = np.array(emb1)
    emb2 = np.array(emb2)
    cos_distance = 1 - np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return cos_distance

def find_closest_match(detected_embedding, student_db):
    min_distance = float('inf')
    matched_name = "Unknown"
    for name, embedding in student_db.items():
        distance = calculate_cosine_distance(detected_embedding, embedding)
        if distance < min_distance:
            min_distance = distance
            matched_name = name
    return matched_name, min_distance

def match_faces(face_encodings, student_db, tolerance=0.6):
    attendance = {name: "Absent" for name in student_db.keys()}
    for encoding in face_encodings:
        matches = face_recognition.compare_faces(list(student_db.values()), encoding, tolerance=tolerance)
        face_distances = face_recognition.face_distance(list(student_db.values()), encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = list(student_db.keys())[best_match_index]
            attendance[name] = "Present"
        else:
            print("Unknown face detected.")
    return attendance

def extract_face_embeddings(classroom_image_path, detections):
    image = cv2.imread(classroom_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_encodings = []
    face_locations = []

    for key, detection in detections.items():
        facial_area = detection["facial_area"]  # [x1, y1, x2, y2]
        x1, y1, x2, y2 = facial_area
        face_image = image_rgb[y1:y2, x1:x2]
        encodings = face_recognition.face_encodings(face_image)
        if encodings:
            face_encodings.append(encodings[0])
            face_locations.append((y1, x2, y2, x1))  # Convert to (top, right, bottom, left)
        else:
            print(f"No encoding found for face {key}.")
    return face_encodings, face_locations, image

def annotate_image(image, detections, face_locations, face_names, output_path='annotated_classroom_retinaface.jpg'):
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a rectangle around the face
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)

        # Label the face
        cv2.rectangle(image, (left, bottom - 25), (right, bottom), (0, 255, 0), cv2.FILLED)
        cv2.putText(image, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Save the annotated image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved as {output_path}.")

def save_attendance_to_csv(attendance, csv_filename='attendance_retinaface.csv'):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M:%S")
    data = []
    for name, status in attendance.items():
        data.append({"Name": name, "Status": status, "Timestamp": date_time})

    df = pd.DataFrame(data)
    df.to_csv(csv_filename, index=False)
    print(f"Attendance has been recorded in {csv_filename}.")

def main():
    student_directory = 'students/'
    classroom_image_path = 'classroom.jpg'

    # Step 1: Prepare the student database
    print("Preparing the student database...")
    student_db = prepare_student_database(student_directory)

    if not student_db:
        print("No student encodings found. Exiting.")
        return

    # Step 2: Detect faces in the classroom image
    print("Detecting faces in the classroom image...")
    detections = detect_faces_in_classroom(classroom_image_path)

    if not detections:
        print("No faces detected in the classroom image. Exiting.")
        return

    # Step 3: Extract face encodings
    print("Extracting face encodings from detected faces...")
    face_encodings, face_locations, image = extract_face_embeddings(classroom_image_path, detections)

    if not face_encodings:
        print("No face encodings extracted. Exiting.")
        return

    # Step 4: Match detected faces with the student database
    print("Matching detected faces with the student database...")
    attendance = match_faces(face_encodings, student_db)

    # Step 5: Annotate the classroom image
    print("Annotating the classroom image...")
    face_names = []
    for encoding in face_encodings:
        matches = face_recognition.compare_faces(list(student_db.values()), encoding, tolerance=0.6)
        face_distances = face_recognition.face_distance(list(student_db.values()), encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = list(student_db.keys())[best_match_index]
            face_names.append(name)
        else:
            face_names.append("Unknown")

    annotate_image(image, detections, face_locations, face_names, output_path='annotated_classroom_retinaface.jpg')

    # Step 6: Save attendance to CSV
    print("Saving attendance to CSV...")
    save_attendance_to_csv(attendance, csv_filename='attendance_retinaface.csv')

    # Optional: Display attendance
    print("\nAttendance Report:")
    for name, status in attendance.items():
        print(f"{name}: {status}")

if __name__ == "__main__":
    main()
